In [1]:
import pandas as pd
import pycountry 
import math
import numpy as np
import difflib
#import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
#from scipy.stats.stats import pearsonr

In [2]:
df = df = pd.read_csv('matches_with_gender.csv', index_col = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186128 entries, 0 to 186127
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          186073 non-null  object 
 1   tourney_name        186103 non-null  object 
 2   surface             185940 non-null  object 
 3   draw_size           186099 non-null  float64
 4   tourney_level       186099 non-null  object 
 5   tourney_date        186100 non-null  float64
 6   match_num           186101 non-null  float64
 7   winner_id           186073 non-null  float64
 8   winner_entry        25827 non-null   object 
 9   winner_name         186101 non-null  object 
 10  winner_hand         186082 non-null  object 
 11  winner_ht           49341 non-null   float64
 12  winner_ioc          186099 non-null  object 
 13  winner_age          183275 non-null  float64
 14  loser_id            186100 non-null  float64
 15  loser_entry         44154 non-null

## wrong values check

### tourney_name

id con più nomi associati

In [3]:
df['tourney_name']=df['tourney_name'].str.upper()
#df['tourney_id']= df['tourney_id'].str[5:]
df['tourney_name']=df['tourney_name'].replace({' CH': '', '  CH': '', "'": '', "-": '', ' MASTERS':'','  ':' ','  ':' '}, regex=True)
TourneyId_name = df.groupby(['tourney_id']).tourney_name.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
TourneyId_name = TourneyId_name[TourneyId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.tourney_id.isin(TourneyId_name.index)].groupby(['tourney_id']).tourney_name.unique()



tourney_id
2016-O16                            [OLYMPICS, RIO OLYMPICS]
2017-M015                                [BEIJING , BEIJING]
2018-M010                  [S HERTOGENBOSCH, SHERTOGENBOSCH]
2019-M010                  [S HERTOGENBOSCH, SHERTOGENBOSCH]
2021-581     [DOHA AUS OPEN QUALIES, DUBAI AUS OPEN QUALIES]
Name: tourney_name, dtype: object

In [4]:
df['tourney_name'] = df['tourney_name'].str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')

df['tourney_name']=df['tourney_name'].replace( {
    'DOHA AUS OPEN QUALIES': 'AUS OPEN QUALIES',
    'DUBAI AUS OPEN QUALIES': 'AUS OPEN QUALIES',
    'SHERTOGENBOSCH': 'S HERTOGENBOSCH',
    'BEIJING ': 'BEIJING',
    'RIO OLYMPICS':'OLYMPICS'
}, regex=True)



TourneyId_name = df.groupby(['tourney_id']).tourney_name.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
TourneyId_name = TourneyId_name[TourneyId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.tourney_id.isin(TourneyId_name.index)].groupby(['tourney_id','tourney_name']).count()


,,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,...,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue,winner_gender,loser_gender
tourney_id,tourney_name,,,,,,,,,,,,,,,,,,,,,


In [5]:
df['tourney_id']= df['tourney_id'].str[5:]
TourneyId_name = df.groupby(['tourney_id']).tourney_name.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
TourneyId_name = TourneyId_name[TourneyId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.tourney_id.isin(TourneyId_name.index)].groupby(['tourney_id']).tourney_name.unique()

tourney_id
0405              [TOKYO , JAPAN OPEN TOKYO, HIROSHIMA]
0568                    [ST. PETERSBURG, ST PETERSBURG]
0637                       [CHAMPAIGN, CHAMPAIGNURBANA]
0703                               [STANFORD, SAN JOSE]
0741                           [EASTBOURNE, NOTTINGHAM]
0784                                 [SEVILLE, SEVILLA]
0806                      [TORONTO, MONTREAL, TORONTO ]
0808                       [SINGAPORE, SHENZHEN FINALS]
0891                                    [PUNE, CHENNAI]
1056                                     [TOKYO, OSAKA]
1091                                 [CALVIA, MALLORCA]
1094                                 [GSTAAD, LAUSANNE]
1096                                  [TAIPEI, HUA HIN]
1796              [ST. BRIEUC, SAINT BRIEUC, ST.BRIEUC]
2000                             [CHICAGO, CHICAGO 125]
2151                              [ROME, ROME , ROME 1]
3618                   [ASTANA, NURSULTAN, NURSULTAN 1]
3627                                 

In [20]:

df['tourney_name']=df['tourney_name'].replace( {
    ' 1': '',
    ' 2': '',
    ' 125': '',
    '25':'',
    'MASTERS ':'',
    'MASTERS  ':'',
    ' MASTERS ':'',
#    " ":"",
    'CH ':'',
    'FINALS':'',
    'SAN ':'ST. ',
    'SAINT ':'ST. ',
    'ST ':'ST. ',
 #   '[.]':''
    
}, regex=True)


#df['tourney_name']=df['tourney_name'].replace(to_replace=''/^[a-z]+$/i', value='', regex=True)

TourneyId_name = df.groupby(['tourney_id']).tourney_name.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
TourneyId_name = TourneyId_name[TourneyId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

print(df[df.tourney_id.isin(TourneyId_name.index)].groupby(['tourney_id']).tourney_name.unique())
print(df[df.tourney_id.isin(TourneyId_name.index)].groupby(['tourney_id']).tourney_name.unique().count())

tourney_id
0405              [TOKYO , JAPAN OPEN TOKYO, HIROSHIMA]
0637                       [CHAMPAIGN, CHAMPAIGNURBANA]
0703                               [STANFORD, ST. JOSE]
0741                           [EASTBOURNE, NOTTINGHAM]
0784                                 [SEVILLE, SEVILLA]
0806                      [TORONTO, MONTREAL, TORONTO ]
0808                             [SINGAPORE, SHENZHEN ]
0891                                    [PUNE, CHENNAI]
1056                                     [TOKYO, OSAKA]
1091                                 [CALVIA, MALLORCA]
1094                                 [GSTAAD, LAUSANNE]
1096                                  [TAIPEI, HUA HIN]
1796                            [ST. BRIEUC, ST.BRIEUC]
2151                                      [ROME, ROME ]
3618                                [ASTANA, NURSULTAN]
3627                                    [TODI, LAQUILA]
3945          [ST. BENEDETTO DEL TRONTO, ST. BENEDETTO]
4012                              [TA

nomi dei tornei con più id

In [7]:
TourneyId_name = df.groupby(['tourney_name']).tourney_id.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
#TourneyId_name.loc[TourneyId_name['tourney_name' == "OBIDOS $25K"]] # it contains the TourneyId values associated with more than one Tourneyname value

#print(TourneyId_name)

df.loc[df['tourney_name'] == 'ST. BRIEUC', 'tourney_id']



68786     1796
68787     1796
68788     1796
68789     1796
68790     1796
          ... 
102143    1796
102144    1796
102145    1796
102146    1796
102147    1796
Name: tourney_id, Length: 108, dtype: object

## winner_name and loser_name


cocatenare senza spazio 

In [8]:
df['winner_name']=df['winner_name'].str.upper()
#df['winner_name']=df['winner_name'].replace({' ':'','  ':''}, regex=True)
WinnerId_name = df.groupby(['winner_name']).winner_id.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
WinnerId_name = WinnerId_name[WinnerId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.winner_name.isin(WinnerId_name.index)].groupby(['winner_name']).winner_id.unique()



winner_name
ASTRID CIROTTE             [236980.0, 223401.0]
FITRIANI SABATINI          [221189.0, 239429.0]
KUAN YI LEE                [134120.0, 221745.0]
LIISA VARUL                [222914.0, 222915.0]
MARIA FERNANDA NAVARRO     [223367.0, 223287.0]
NANA KAWAGISHI             [221771.0, 244078.0]
SOFIA NAHIARA GARCIA       [220928.0, 236977.0]
STANISLAVA BOBROVNIKOVA    [221347.0, 225865.0]
VALENTINA LIA              [220334.0, 239415.0]
Name: winner_id, dtype: object

In [9]:
df['loser_name']=df['loser_name'].str.upper()
#df['loser_name']=df['loser_name'].replace({' ':'','  ':''}, regex=True)
LoserId_name = df.groupby(['loser_name']).loser_id.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
LoserId_name = LoserId_name[LoserId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.loser_name.isin(LoserId_name.index)].groupby(['loser_name']).loser_id.unique()



loser_name
ASTRID CIROTTE             [223401.0, 236980.0]
EKATERINA MAKAROVA         [201505.0, 223126.0]
ELENI FASOULA              [222942.0, 222943.0]
ELLIE MYERS                [222407.0, 240168.0]
EMILSE LUJAN RUIZ          [222646.0, 236974.0]
FITRIANI SABATINI          [221189.0, 239429.0]
GIUSEPPE TRESCA            [206834.0, 207982.0]
GUY STOKMAN                [206883.0, 207142.0]
HEI CHING CLAUDIA NG       [219835.0, 216586.0]
HOLLY FISCHER              [221142.0, 222228.0]
KUAN YI LEE                [134120.0, 221745.0]
LARA ONAL                  [224486.0, 223215.0]
LIISA VARUL                [222915.0, 222914.0]
MARIA FERNANDA NAVARRO     [223367.0, 223287.0]
MATYLDA BURYLO             [222845.0, 222066.0]
NANA KAWAGISHI             [221771.0, 244078.0]
SOFIA NAHIARA GARCIA       [220928.0, 236977.0]
STANISLAVA BOBROVNIKOVA    [221347.0, 225865.0]
VALENTINA LIA              [220334.0, 239415.0]
Name: loser_id, dtype: object

questi sono i nomi dei giocatori in cui ho associato più di un id per ciascuno/a
    
    ASTRID CIROTTE             [223401.0, 236980.0]
    EKATERINA MAKAROVA         [201505.0, 223126.0]   
    FITRIANI SABATINI          [221189.0, 239429.0]
    ELENI FASOULA              [222942.0, 222943.0]
    ELLIE MYERS                [222407.0, 240168.0]
    EMILSE LUJAN RUIZ          [222646.0, 236974.0]
    GIUSEPPE TRESCA            [206834.0, 207982.0]
    GUY STOKMAN                [206883.0, 207142.0]
    HEI CHING CLAUDIA NG       [219835.0, 216586.0]
    HOLLY FISCHER              [221142.0, 222228.0]
    KUAN YI LEE                [134120.0, 221745.0]
    LARA ONAL                  [224486.0, 223215.0]
    LIISA VARUL                [222915.0, 222914.0]
    MARIA FERNANDA NAVARRO     [223367.0, 223287.0]
    MATYLDA BURYLO             [222845.0, 222066.0]
    NANA KAWAGISHI             [221771.0, 244078.0]
    SOFIA NAHIARA GARCIA       [220928.0, 236977.0]
    STANISLAVA BOBROVNIKOVA    [221347.0, 225865.0]
    VALENTINA LIA              [220334.0, 239415.0]

adesso tra questi trovo quello con id più frequente

In [10]:
def compareAndSum(first,second):
    a=df[df['loser_id'] == first]['loser_id'].count()+df[df['winner_id'] == first]['winner_id'].count()
    b=df[df['loser_id'] == second]['loser_id'].count()+df[df['winner_id'] == second]['winner_id'].count()
    
    if a<b:
        return second
    else:
        return first

def changevalue(x,y,v):
    df['loser_id']=df['loser_id'].replace({x:v}, regex=True)
    df['winner_id']=df['winner_id'].replace({x:v}, regex=True)
    df['loser_id']=df['loser_id'].replace({y:v}, regex=True)
    df['winner_id']=df['winner_id'].replace({y:v}, regex=True)
    

In [11]:
#ASTRID CIROTTE             [223401.0, 236980.0]
values=compareAndSum(223401.0,236980.0)
changevalue(223401.0,236980.0,values)

#EKATERINA MAKAROVA         [201505.0, 223126.0]
values=compareAndSum(201505.0,223126.0)
changevalue(201505.0,223126.0,values)

#ELENI FASOULA              [222942.0, 222943.0]
values=compareAndSum(222942.0,222943.0)
changevalue(222942.0,222943.0,values)

#FITRIANI SABATINI          [221189.0, 239429.0]
values=compareAndSum(221189.0,239429.0)
changevalue(221189.0,239429.0,values)

##ELLIE MYERS                [222407.0, 240168.0]
values=compareAndSum(222407.0,240168.0)
changevalue(222407.0,240168.0,values)

#EMILSE LUJAN RUIZ          [222646.0, 236974.0]
values=compareAndSum(222646.0,236974.0)
changevalue(222646.0,236974.0,values)

#GIUSEPPE TRESCA            [206834.0, 207982.0]
values=compareAndSum(206834.0,207982.0)
changevalue(206834.0,207982.0,values)

#GUY STOKMAN                [206883.0, 207142.0]
values=compareAndSum(206883.0,207142.0)
changevalue(206883.0,207142.0,values)

#HEI CHING CLAUDIA NG       [219835.0, 216586.0]
values=compareAndSum(219835.0,216586.0)
changevalue(219835.0,216586.0,values)

#HOLLY FISCHER              [221142.0, 222228.0]
values=compareAndSum(221142.0,222228.0)
changevalue(221142.0,222228.0,values)

#KUAN YI LEE                [134120.0, 221745.0]
values=compareAndSum(134120.0,221745.0)
changevalue(134120.0,221745.0,values)

#LARA ONAL                  [224486.0, 223215.0]
values=compareAndSum(224486.0,223215.0)
changevalue(224486.0,223215.0,values)

#LIISA VARUL                [222915.0, 222914.0]
values=compareAndSum(222915.0,222914.0)
changevalue(222915.0,222914.0,values)

#MARIA FERNANDA NAVARRO     [223367.0, 223287.0]
values=compareAndSum(223367.0,223287.0)
changevalue(223367.0,223287.0,values)

#MATYLDA BURYLO             [222845.0, 222066.0]
values=compareAndSum(222845.0,222066.0)
changevalue(222845.0,222066.0,values)

#NANA KAWAGISHI             [221771.0, 244078.0]
values=compareAndSum(221771.0,244078.0)
changevalue(221771.0,244078.0,values)

#SOFIA NAHIARA GARCIA       [220928.0, 236977.0]
values=compareAndSum(220928.0,236977.0)
changevalue(220928.0,236977.0,values)

#STANISLAVA BOBROVNIKOVA    [221347.0, 225865.0]
values=compareAndSum(221347.0,225865.0)
changevalue(221347.0,225865.0,values)

#VALENTINA LIA              [220334.0, 239415.0]
values=compareAndSum(220334.0,239415.0)
changevalue(220334.0,239415.0,values)


In [12]:

LoserId_name = df.groupby(['loser_name']).loser_id.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
LoserId_name = LoserId_name[LoserId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.loser_name.isin(LoserId_name.index)].groupby(['loser_name']).loser_id.unique()



Series([], Name: loser_id, dtype: float64)

In [13]:
WinnerId_name = df.groupby(['winner_name']).winner_id.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
WinnerId_name = WinnerId_name[WinnerId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.winner_name.isin(WinnerId_name.index)].groupby(['winner_name']).winner_id.unique()


Series([], Name: winner_id, dtype: float64)

In [14]:

LoserId_name = df.groupby(['loser_id']).loser_name.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
LoserId_name = LoserId_name[LoserId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.loser_id.isin(LoserId_name.index)].groupby(['loser_id']).loser_name.unique().count()





48

In [15]:

WinnerId_name = df.groupby(['winner_id']).winner_name.nunique() # it contains each value of TourneyId and the number of Tourneyname values it is associated with 
WinnerId_name = WinnerId_name[WinnerId_name > 1] # it contains the TourneyId values associated with more than one Tourneyname value

df[df.winner_id.isin(WinnerId_name.index)].groupby(['winner_id']).winner_name.unique().count()




13

## winner_ioc

Analisi: tutte le nazioni esistono correttamente

In [16]:
#numero di nazioni diverse e numero di volte che si ripetono
country_winner_dim=df.groupby(df.winner_ioc,as_index=False).size()
#country_loser_dim=df.groupby(df.loser_ioc,as_index=False).size()

nation=np.concatenate((df.winner_ioc.unique(), df.loser_ioc.unique()),0)
country = pd.DataFrame(nation)
country = country.drop_duplicates()

no=0

for i in country[0]:
    try:
        pycountry.countries.get(alpha_3=i)
    except:
        no+=1
        print(i)
        
print("no:",no)


nan
no: 1
